# Part III: Gene Set Enrichment Analysis

## 1. Load the required packages

In [ ]:
library(MOFA2)
library(msigdbr)
library(data.table)
library(tibble)
library(stringr)

## 2. Load the saved MOFA

In [ ]:
serialized_mofa_file = file.path(getwd(), "../MOFAobject.LUAD.hdf5")
MOFAobject.trained <- load_model(serialized_mofa_file)
views_names(MOFAobject.trained) <- c("CNV","Protein","RNA")

## 3. Load and explore MSIGDB data

In [ ]:
msigdbr(species = "Homo sapiens")
msigdbr_collections() %>% data.frame

# Extract specific databases as binary matrix
msigdb.hallmark <- msigdbr(
  species = "Homo sapiens",
  category = "H"
  ) %>% as.data.table %>% .[,id:=1] %>%
  dcast(gs_name~gene_symbol, value.var="id", fill=0) %>%
  column_to_rownames("gs_name") %>% 
  as.matrix()

msigdb.reactome <- msigdbr(
  species = "Homo sapiens",
  category = "C2", 
  subcategory = "CP:REACTOME"
  ) %>% as.data.table %>% .[,id:=1] %>%
  dcast(gs_name~gene_symbol, value.var="id", fill=0) %>%
  column_to_rownames("gs_name") %>%
  as.matrix()

msigdb.gobp <- msigdbr(
  species = "Homo sapiens",
  category = "C5", 
  subcategory = "GO:BP"
  ) %>% as.data.table %>% .[,id:=1] %>%
  dcast(gs_name~gene_symbol, value.var="id", fill=0) %>%
  column_to_rownames("gs_name") %>% 
  as.matrix()

# Convert to binary matrix --- removes duplicate genes
msigdb.hallmark[which(msigdb.hallmark > 1)] <-  1
msigdb.reactome[which(msigdb.reactome > 1)] <-  1
msigdb.gobp[which(msigdb.gobp > 1)] <-  1

# Rename gene names to be identical with MOFA feature names
# This is layer-specific --> Need to redo for protein and CNV, too
colnames(msigdb.hallmark) <- str_c("rna_",colnames(msigdb.hallmark)) 
colnames(msigdb.reactome) <- str_c("rna_",colnames(msigdb.reactome)) 
colnames(msigdb.gobp) <- str_c("rna_",colnames(msigdb.gobp))

## 4. Run *Gene Set Enrichment Analysis*

In [ ]:
# Hallmark on positive weights
gsea.hallmark.positive <- run_enrichment(MOFAobject.trained, 
  feature.sets = msigdb.hallmark, 
  view = "RNA",
  sign = "positive"
)

# Hallmark on negative weights
gsea.hallmark.negative <- run_enrichment(MOFAobject.trained, 
  feature.sets = msigdb.hallmark, 
  view = "RNA",
  sign = "negative"
)

# TODO: Repeat for reactome and GOBD

### 4.1 Visualize results

In [ ]:
# Simple GSEA overview per factor, layer and directionality
plot_enrichment(gsea.hallmark.positive, factor=1)
plot_enrichment(gsea.hallmark.negative, factor=1)

# Detailed GSEA overview per factor, layer and directionality
plot_enrichment_detailed(gsea.hallmark.positive, factor=1)
plot_enrichment_detailed(gsea.hallmark.negative, factor=1)

# GSEA heatmap
# This method generates a heatmap with the adjusted p.values that result from the feature set enrichment analysis. Rows are feature sets and columns are factors.
plot_enrichment_heatmap(gsea.hallmark.positive)
plot_enrichment_heatmap(gsea.hallmark.negative)

# Write enrichment values to csv files
write.csv(as.data.frame(gsea.hallmark.positive$pval.adj), paste0("../", format(Sys.time(), "%d-%b-%Y %H.%M"), "gsea.hallmark.positive.pvals", ".csv"))
write.csv(as.data.frame(gsea.hallmark.negative$pval.adj), paste0("../", format(Sys.time(), "%d-%b-%Y %H.%M"), "gsea.hallmark.negative.pvals", ".csv"))